# Perfil de comércio

## Setup

### Criar directorias de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

### Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = 
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


### Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


### Inicializar a API e transferir os códigos usados

In [3]:
import os
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    # get API Key or set to None    
    APIKEY = config['comtrade'].get('key', None)

comtrade.init(APIKEY, force_init=True)
comtrade.CALL_PER_PERIOD=10  # Period default is 60 seconds


contrade.py initialized


## Country and period of interest

Choose country and years of interest

In [4]:
country_of_interest = comtrade.m49_angola
country_of_interest_desc = comtrade.COUNTRY_CODES[country_of_interest]
years_of_interest = comtrade.year_range(2019, 2021)   
print(country_of_interest_desc, years_of_interest)

Angola 2019,2020,2021


## Trade balance

In [5]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format


global_trade = comtrade.get_global_stats(country_of_interest, period=years_of_interest,
                                         exports_from_world_imports=True)


In [6]:
global_trade

flowDesc,countryCode,countryDesc,exports,imports,trade_balance,xReporterCode,xReporterDesc
refYear,,,,,,,
2019,24,Angola,"37,541,664,634.40","13,961,640,425.76","23,580,024,208.64",0,World
2020,24,Angola,"24,009,441,917.40","9,337,925,708.64","14,671,516,208.76",0,World
2021,24,Angola,"33,477,533,120.02","11,378,927,920.70","22,098,605,199.32",0,World


## Exports

Usamos as importações declaradas pelos parceiros para obter as
exportações do país de interesse.

### Main export partners of the country

In [7]:

# We get the exports from the reported imports of all the countries
# i.e. instead of asking Angola for export data we ask the world for import data from Angola

years_of_interest = comtrade.year_range(2011, 2021)  
df = comtrade.get_data(reporterCode=None,
                       typeCode='C',  # commodities
                       freqCode='A',  # annual
                       partnerCode=country_of_interest,
                       period=years_of_interest,
                       motCode=0,  # all means of transport
                       flowCode='M',
                       timeout=120)

/workspaces/cipf-comtrade/comtrade.py:370: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  warnings.warn("Query returned different isAggregate values, check for duplicate results when aggregating")


In [8]:
import pandas as pd
# show 500 lines
pd.options.display.max_rows=500
pd.options.display.float_format='{:,.2f}'.format

show_only_top = 10 
cols=['refYear','flowCode','reporterDesc','cmdCode','partner_upper_sum','partner_sum','partner_rank','partner_perc','partner_upper_perc']
formats =  {
           'partner_perc': '{0:.3%}',
           'partner_upper_perc': '{0:.3%}',
           'partner_sum': '${0:,.0f}',
           'partner_upper_sum': '${0:,.0f}',
           }
# aggregate by year, flow, reporter 
exports = comtrade.total_rank_perc(df,groupby=['refYear','flowCode','reporterDesc'],
                                   col='primaryValue',
                                   prefix='partner',
                                   drop_duplicates=True)

top_export_partners = exports[exports['partner_rank']<=show_only_top]['reporterDesc'].unique()
top_export_partners_codes = exports[exports['partner_rank']<=show_only_top]['reporterCode'].unique()

print(f"Top {show_only_top} export partners of {country_of_interest_desc} in {years_of_interest}: \n{top_export_partners}")
exports[exports['partner_rank']<=show_only_top][cols].sort_values(['refYear','flowCode','partner_rank','reporterDesc','cmdCode']).style.format(formats)

Top 10 export partners of Angola in 2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021: 
['Canada' 'China' 'France and Monaco' 'Germany' 'Italy'
 'Asia, not elsewhere specified' 'Portugal' 'India' 'South Africa'
 'United States of America' 'Congo' 'Spain' 'United Kingdom' 'Netherlands'
 'United Arab Emirates' 'Belgium' 'Indonesia' 'Thailand' 'Singapore'
 'Chile']


,refYear,flowCode,reporterDesc,cmdCode,partner_upper_sum,partner_sum,partner_rank,partner_perc,partner_upper_perc
23,2011,M,China,TOTAL,"$66,204,920,205","$24,922,180,492",1,37.644%,37.644%
170,2011,M,United States of America,TOTAL,"$66,204,920,205","$13,833,012,040",2,20.894%,20.894%
117,2011,M,India,TOTAL,"$66,204,920,205","$6,005,484,849",3,9.071%,9.071%
87,2011,M,"Asia, not elsewhere specified",TOTAL,"$66,204,920,205","$5,665,315,729",4,8.557%,8.557%
20,2011,M,Canada,TOTAL,"$66,204,920,205","$2,469,608,910",5,3.730%,3.730%
70,2011,M,Italy,TOTAL,"$66,204,920,205","$2,068,356,894",6,3.124%,3.124%
37,2011,M,France and Monaco,TOTAL,"$66,204,920,205","$1,825,704,019",7,2.758%,2.758%
101,2011,M,Portugal,TOTAL,"$66,204,920,205","$1,639,710,523",8,2.477%,2.477%
122,2011,M,South Africa,TOTAL,"$66,204,920,205","$1,586,176,197",9,2.396%,2.396%
41,2011,M,Germany,TOTAL,"$66,204,920,205","$1,192,877,561",10,1.802%,1.802%


Passamos os países para colunas, mantendo a percentagem e o valor em separado.

In [9]:
pivoted = df.pivot_table(index=['refYear','flowDesc'], columns='reporterDesc', values=['primaryValue','partner_perc'])
pivoted.swaplevel(axis=1).fillna('',inplace=True)
pivoted.sort_index(axis=1, level=0, inplace=True)
pivoted.fillna(0)

partner_perc                                               \
reporterDesc          Albania Algeria Angola Antigua and Barbuda Argentina   
refYear flowDesc                                                             
2011    Import           0.00    0.00   0.00                0.00      0.00   
2012    Import           0.00    0.00   0.00                0.00      0.00   
2013    Import           0.00    0.00   0.00                0.00      0.00   
2014    Import           0.00    0.00   0.00                0.00      0.00   
2015    Import           0.00    0.00   0.00                0.00      0.00   
2016    Import           0.00    0.00   0.00                0.00      0.00   
2017    Import           0.00    0.00   0.00                0.00      0.00   
2018    Import           0.00    0.00   0.00                0.00      0.00   
2019    Import           0.00    0.00   0.00                0.00      0.00   
2020    Import           0.00    0.00   0.00                0.00      0.00   
2021    Import           0.00    0.00   0.00                0.00      0.00   

                                                                          \
reporterDesc     Armenia Asia, not elsewhere specified Australia Austria   
refYear flowDesc                                                           
2011    Import      0.00                          0.09      0.00    0.00   
2012    Import      0.00                          0.07      0.00    0.00   
2013    Import      0.00                          0.05      0.00    0.00   
2014    Import      0.00                          0.05      0.00    0.00   
2015    Import      0.00                          0.04      0.00    0.00   
2016    Import      0.00                          0.05      0.00    0.00   
2017    Import      0.00                          0.04      0.00    0.00   
2018    Import      0.00                          0.01      0.00    0.00   
2019    Import      0.00                          0.01      0.00    0.00   
2020    Import      0.00                          0.02      0.00    0.00   
2021    Import      0.00                          0.01      0.00    0.00   

                             ... primaryValue                       \
reporterDesc     Azerbaijan  ...      Ukraine United Arab Emirates   
refYear flowDesc             ...                                     
2011    Import         0.00  ...   146,120.00       301,582,058.00   
2012    Import         0.00  ...   764,720.00     1,234,968,772.00   
2013    Import         0.00  ...    63,422.00       757,063,814.73   
2014    Import         0.00  ...    11,069.00       812,542,196.16   
2015    Import         0.00  ...    50,320.00     1,056,613,911.80   
2016    Import         0.00  ...   299,907.11       956,305,699.01   
2017    Import         0.00  ...    72,378.45     1,200,168,189.47   
2018    Import         0.00  ...    20,096.32     1,569,493,453.56   
2019    Import         0.00  ...   251,112.36     1,294,307,774.80   
2020    Import         0.00  ...    35,298.67     1,018,255,600.93   
2021    Import         0.00  ...     1,389.42     1,685,701,656.52   

                                                                           \
reporterDesc       United Kingdom United States of America        Uruguay   
refYear flowDesc                                                            
2011    Import     437,170,828.00        13,833,012,040.00          16.00   
2012    Import   1,261,394,647.00        10,030,264,550.00           0.00   
2013    Import     994,286,997.00         8,922,894,861.00 315,549,401.00   
2014    Import     650,319,421.00         5,837,752,018.00 217,485,863.00   
2015    Import   1,006,251,202.00         2,917,054,295.00       1,019.00   
2016    Import     369,342,914.00         2,976,766,381.00 226,117,355.00   
2017    Import      71,778,081.35         2,686,493,698.00           0.00   
2018    Import      83,063,568.65         2,768,924,982.00 363,283,051.00   
2019    Import      17,811,494.

In [10]:
show_only_columns = [(l1, country) for l1,country in pivoted.columns.tolist() if country in top_export_partners]
pivoted[show_only_columns]

partner_perc                             \
reporterDesc     Asia, not elsewhere specified Belgium Canada Chile China   
refYear flowDesc                                                            
2011    Import                            0.09    0.00   0.04  0.00  0.38   
2012    Import                            0.07    0.01   0.03  0.00  0.44   
2013    Import                            0.05    0.01   0.02  0.00  0.44   
2014    Import                            0.05    0.01   0.02  0.01  0.48   
2015    Import                            0.04    0.02   0.01  0.00  0.43   
2016    Import                            0.05    0.02   0.00  0.00  0.49   
2017    Import                            0.04    0.01   0.00  0.00  0.56   
2018    Import                            0.01    0.01   0.00  0.00  0.58   
2019    Import                            0.01    0.01   0.00  0.01  0.63   
2020    Import                            0.02    0.01   0.00  0.00  0.61   
2021    Import                            0.01    0.02   0.00  0.02  0.62   

                                                                  ...  \
reporterDesc     Congo France and Monaco Germany India Indonesia  ...   
refYear flowDesc                                                  ...   
2011    Import    0.01              0.03    0.02  0.09      0.00  ...   
2012    Import    0.02              0.02    0.00  0.11      0.01  ...   
2013    Import    0.02              0.02    0.01  0.09      0.00  ...   
2014    Import    0.00              0.03    0.00  0.09      0.00  ...   
2015    Import    0.01              0.04    0.00  0.09      0.02  ...   
2016    Import    0.00              0.04    0.01  0.07      0.01  ...   
2017    Import    0.01              0.01    0.00  0.11      0.01  ...   
2018    Import    0.00              0.01    0.00  0.10      0.02  ...   
2019    Import    0.00              0.01    0.00  0.10      0.00  ...   
2020    Import    0.00              0.02    0.00  0.08      0.00  ...   
2021    Import    0.00              0.01    0.00  0.07      0.01  ...   

                     primaryValue                                    \
reporterDesc                Italy      Netherlands         Portugal   
refYear flowDesc                                                      
2011    Import   2,068,356,894.00   933,906,994.74 1,639,710,523.00   
2012    Import     877,520,110.00   737,821,301.70 2,290,610,271.00   
2013    Import     720,623,566.00 1,417,732,100.43 3,495,088,409.00   
2014    Import   1,232,641,645.00 1,219,004,294.36 2,132,511,992.00   
2015    Import     971,600,225.84   512,901,212.71 1,266,536,955.09   
2016    Import     452,291,731.41   172,254,148.00   895,881,332.31   
2017    Import     390,885,197.89     5,327,723.16   315,119,621.26   
2018    Import     522,644,162.22    32,217,465.81 1,173,908,104.09   
2019    Import     528,525,814.14    65,681,447.67 1,204,116,161.11   
2020    Import     282,787,004.45   289,511,616.01   444,757,098.52   
2021    Import     140,589,239.50   300,018,788.20    95,324,574.46   

                                                                   \
reporterDesc          Singapore     South Africa            Spain   
refYear flowDesc                                                    
2011    Import     3,130,004.09 1,586,176,196.93   587,380,397.71   
2012    Import       891,617.19 2,802,783,009.10 1,424,748,647.92   
2013    Import   144,463,922.25 1,958,574,119.85 2,883,313,128.47   
2014    Import   105,426,901.25 2,011,018,978.76 3,616,693,014.01   
2015    Import     1,875,945.57 1,344,214,553.60 2,241,044,685.06   
2016    Import    81,406,154.24 1,277,795,554.60   910,706,245.08   
2017    Import   104,446,151.90 1,340,980,393.36 1,070,297,304.75   
2018    Import    73,371,402.22 1,282,900,382.26 1,425,408,792.80   
2019    Import   272,207,226.60   563,803,802.43 1,073,792,688.36   
2020    Import   396,399,839.00    60,149,788.46   626,580,515.28   
2021    Import   752,550,761.40   361,85

#### Export to Excel

In [11]:
exports.to_excel(f'{country_of_interest_desc}_top_partners.xlsx')
pivoted.to_excel(f'{country_of_interest_desc}_top_partners_cols.xlsx')
pivoted[show_only_columns].to_excel(f'{country_of_interest_desc}_export_top_{show_only_top}_partners_cols.xlsx')

## Exports: products

### Main exported products

In [12]:
import pandas as pd

# show 500 lines of dataframes
pd.set_option('display.max_rows', 500)

show_only_top=5
years_of_interest = comtrade.year_range(2020, 2021)  


Get the products imported from Angola

In [13]:
df = comtrade.get_data(reporterCode=None,
                       typeCode='C',  # commodities
                       freqCode='A',  # annual
                       partnerCode=country_of_interest,
                       period=years_of_interest,
                       cmdCode='AG2',  # Top level product types
                       motCode=0,  # all means of transport
                       flowCode='M',
                       timeout=120)

In [14]:
import pandas as pd
# show 500 lines
pd.options.display.max_rows=500
pd.options.display.float_format='{:,.2f}'.format

formats =  {
           'product_perc': '{0:.3%}',
           'product_upper_perc': '{0:.3%}',
           'primaryValue': '${0:,.0f}',
           'product_sum': '${0:,.0f}',
           'product_upper_sum': '${0:,.0f}',
           }
show_only_top = 5 
exports = comtrade.total_rank_perc(df,['refYear','flowCode','cmdCode'],'primaryValue','product',drop_duplicates=True)
cols=['refYear','flowCode','cmdCode','product_sum','product_rank','product_perc','product_upper_sum','product_upper_perc']

top_export_products_codes = exports[exports['product_rank']<=show_only_top]['cmdCode'].unique()

print(f"Top products exported by {country_of_interest_desc} in {years_of_interest}:\n {top_export_products_codes}")
for cmdCode in top_export_products_codes:
    print(f"   {cmdCode}: {comtrade.HS_CODES.get(cmdCode,cmdCode)}")
exports[exports['product_rank']<=show_only_top][cols].sort_values(['refYear','flowCode','product_rank']).style.format(formats)


Top products exported by Angola in 2020,2021:
 ['25' '27' '71' '84' '03']
   25: Salt; sulphur; earths, stone; plastering materials, lime and cement
   27: Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
   71: Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin
   84: Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof
   03: Fish and crustaceans, molluscs and other aquatic invertebrates


,refYear,flowCode,cmdCode,product_sum,product_rank,product_perc,product_upper_sum,product_upper_perc
10,2020,M,27,"$22,333,815,487",1,0.001%,"$24,009,441,917",93.021%
53,2020,M,71,"$1,411,461,142",2,1.342%,"$24,009,441,917",5.879%
5,2020,M,25,"$51,655,007",3,0.000%,"$24,009,441,917",0.215%
123,2020,M,03,"$36,822,340",4,0.000%,"$24,009,441,917",0.153%
65,2020,M,84,"$27,833,891",5,0.002%,"$24,009,441,917",0.116%
2980,2021,M,27,"$30,623,115,913",1,0.026%,"$33,477,533,120",91.474%
3016,2021,M,71,"$2,480,934,829",2,1.811%,"$33,477,533,120",7.411%
2976,2021,M,25,"$61,619,118",3,0.000%,"$33,477,533,120",0.184%
2968,2021,M,03,"$36,231,724",4,0.000%,"$33,477,533,120",0.108%
3037,2021,M,84,"$34,299,991",5,0.033%,"$33,477,533,120",0.102%


#### Export to Excel

In [15]:
exports.to_excel(f'{country_of_interest_desc}_exports_products.xlsx')

## Export partners / products analysis

What are the partners buying the top exports of country of interest?


In [16]:
import pandas as pd
# show 500 lines
pd.options.display.max_rows=500
pd.options.display.float_format='{:,.2f}'.format

formats =  {
           'partner_cmd_perc': '{0:.3%}',
           'partner_cmd_upper_perc': '{0:.3%}',
           'primaryValue': '${0:,.0f}',
           'partner_cmd_sum': '${0:,.0f}',
           'partner_cmd_upper_sum': '${0:,.0f}',
           }
show_only_top = 5 
partners_and_products = comtrade.total_rank_perc(df,['refYear','flowCode','cmdCode','reporterCode'],'primaryValue','partner_cmd',drop_duplicates=False)
cols=['refYear','flowCode','cmdCode','reporterDesc','partner_cmd_sum','partner_cmd_rank','partner_cmd_upper_sum','partner_cmd_perc']
partners_and_products[(partners_and_products['cmdCode'].isin(top_export_products_codes) 
                       & (partners_and_products['partner_cmd_rank']<=show_only_top))][cols].sort_values(['refYear','flowCode','partner_cmd_upper_sum','partner_cmd_rank'],
                                                                                                        ascending=[True,True,False,True]).style.format(formats)

,refYear,flowCode,cmdCode,reporterDesc,partner_cmd_sum,partner_cmd_rank,partner_cmd_upper_sum,partner_cmd_perc
195,2020,M,27,China,"$14,686,950,196",1,"$22,333,815,487",65.761%
2319,2020,M,27,India,"$2,005,413,125",2,"$22,333,815,487",8.979%
2640,2020,M,27,Thailand,"$1,250,575,334",3,"$22,333,815,487",5.599%
2503,2020,M,27,Spain,"$604,026,343",4,"$22,333,815,487",2.705%
1275,2020,M,27,"Asia, not elsewhere specified","$496,824,440",5,"$22,333,815,487",2.225%
2680,2020,M,71,United Arab Emirates,"$1,015,602,702",1,"$1,411,461,142",71.954%
53,2020,M,71,Belgium,"$322,279,674",2,"$1,411,461,142",22.833%
2398,2020,M,71,South Africa,"$21,913,628",3,"$1,411,461,142",1.553%
310,2020,M,71,China,"$20,014,740",4,"$1,411,461,142",1.418%
2397,2020,M,71,Singapore,"$11,149,551",5,"$1,411,461,142",0.790%




Por cada combinação ano, país parceiro, produto importado, obtém as importações
desse produto pelo parceiro e calcula o ranking do país em análise nas importações
desse produto pelo parceiro.

In [17]:
combos_of_interest = partners_and_products[(partners_and_products['cmdCode'].isin(top_export_products_codes) 
                       & (partners_and_products['partner_cmd_rank']<=show_only_top))][['refYear','reporterCode','cmdCode']].drop_duplicates()
combos_of_interest = [tuple(x) for x in combos_of_interest.values]
combos_of_interest

[(2020, 56, '71'),
 (2020, 156, '25'),
 (2020, 156, '27'),
 (2020, 156, '71'),
 (2020, 178, '84'),
 (2020, 384, '03'),
 (2020, 458, '03'),
 (2020, 380, '25'),
 (2020, 566, '03'),
 (2020, 620, '03'),
 (2020, 616, '25'),
 (2020, 678, '25'),
 (2020, 490, '27'),
 (2020, 516, '84'),
 (2020, 579, '84'),
 (2020, 620, '84'),
 (2020, 699, '27'),
 (2020, 702, '71'),
 (2020, 710, '71'),
 (2020, 724, '03'),
 (2020, 724, '25'),
 (2020, 724, '27'),
 (2020, 764, '27'),
 (2020, 784, '71'),
 (2020, 842, '84'),
 (2021, 56, '71'),
 (2021, 24, '84'),
 (2021, 204, '03'),
 (2021, 156, '25'),
 (2021, 180, '25'),
 (2021, 156, '27'),
 (2021, 178, '84'),
 (2021, 458, '03'),
 (2021, 380, '25'),
 (2021, 620, '03'),
 (2021, 490, '25'),
 (2021, 516, '84'),
 (2021, 579, '84'),
 (2021, 620, '84'),
 (2021, 699, '27'),
 (2021, 702, '27'),
 (2021, 699, '71'),
 (2021, 710, '71'),
 (2021, 724, '03'),
 (2021, 724, '25'),
 (2021, 768, '03'),
 (2021, 764, '27'),
 (2021, 784, '71'),
 (2021, 842, '27'),
 (2021, 842, '71')]

## Get the alternative suppliers for country_of_interest partners



In [18]:
cmdCode_list = [cmdCode for (refYear,countryCode,cmdCode) in combos_of_interest]
cmdCode_list = list(set(cmdCode_list))
countryCode_list = [countryCode for (refYear,countryCode,cmdCode) in combos_of_interest]
countryCode_list = list(set(countryCode_list))
cmdCodeCSV = ','.join(cmdCode_list)
countryCodeCSV = ','.join(map(str,countryCode_list))

In [19]:

df = comtrade.get_data(reporterCode=countryCodeCSV,
                       typeCode='C',  # commodities
                       freqCode='A',  # annual
                       partnerCode=None,
                       period=years_of_interest,
                       cmdCode=cmdCodeCSV,  # Top level product types
                       motCode=0,  # all means of transport
                       flowCode='M',
                       timeout=240)

: 

: 

In [ ]:
df.info()

In [1]:
import pandas as pd
        #  set max rows to 500
pd.set_option('display.max_rows', 1500)

other_suppliers = comtrade.total_rank_perc(df,
                                           groupby=['refYear','flowCode','reporterCode','cmdCode','partnerCode'],
                                           col='primaryValue',
                                           prefix='cmd_partner',
                                           )

cols=['reporterDesc','refYear','cmdCode','cmdDesc','cmd_partner_upper_sum','partnerDesc','cmd_partner_rank','cmd_partner_perc','primaryValue']
other_suppliers[other_suppliers[['refYear','reporterCode','cmdCode']].apply(tuple,axis=1).isin(combos_of_interest)][cols].sort_values(by=['reporterDesc','refYear','cmd_partner_upper_sum','cmd_partner_rank'],
                                                                                                                                      ascending=[True,True,False,True]).head(500)

NameError: name 'comtrade' is not defined

# AQUI

In [ ]:
cofi[cofi.rank_cmd_partner<rank_of_interest]['reporterDesc'].unique()

array(['China', 'United States of America', 'India',
       'Asia, not elsewhere specified', 'Canada', 'Portugal',
       'South Africa', 'Spain', 'France and Monaco',
       'United Arab Emirates'], dtype=object)

In [ ]:
cols=['refYear','reporterDesc','partnerDesc','cmdCode','cmdDesc','rank_cmd_partner','perc_partner_for_cmd','primaryValue']

filter1 = suppliers2.cmdCode.isin(cmd_of_interest)
filter2 = suppliers2.reporterCode.isin(partner_of_interest)
filter3 = (filter1) & (filter2)
suppliers2[(filter3) & (filter2)][cols].sort_values(['refYear','reporterDesc','cmdCode','rank_cmd_partner']).head(100).style.background_gradient(cmap='Blues')


,refYear,reporterDesc,partnerDesc,cmdCode,cmdDesc,rank_cmd_partner,perc_partner_for_cmd,primaryValue
4334,2011,"Asia, not elsewhere specified",China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",1,0.387736,442207162.000000
4249,2011,"Asia, not elsewhere specified",Australia,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2,0.130440,148764450.000000
4699,2011,"Asia, not elsewhere specified",Vietnam,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",3,0.069851,79663854.000000
4493,2011,"Asia, not elsewhere specified",Japan,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",4,0.062368,71129581.000000
4817,2011,"Asia, not elsewhere specified",United States of America,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",5,0.057880,66010783.000000
4773,2011,"Asia, not elsewhere specified",Turkey,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",6,0.041600,47443684.000000
4680,2011,"Asia, not elsewhere specified",India,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",7,0.039385,44917547.000000
4561,2011,"Asia, not elsewhere specified",Mexico,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",8,0.034346,39171435.000000
4282,2011,"Asia, not elsewhere specified",Brazil,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",9,0.032099,36608283.000000
4482,2011,"Asia, not elsewhere specified",Italy,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",10,0.023156,26409468.000000


In [ ]:
suppliers2.to_excel(f'{country_of_interest_desc}_export_partners_alternative.xlsx')

KeyboardInterrupt: 

In [ ]:
cumulative_suppliers.to_excel(f'{country_of_interest_desc}_export_partners_alternative.xlsx')

In [ ]:
cumulative_suppliers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7037 entries, 2 to 638
Data columns (total 65 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   typeCode                  7037 non-null   object 
 1   freqCode                  7037 non-null   object 
 2   refPeriodId               7037 non-null   int64  
 3   refYear                   7037 non-null   int64  
 4   refMonth                  7037 non-null   int64  
 5   period                    7037 non-null   object 
 6   reporterCode              7037 non-null   int64  
 7   reporterISO               0 non-null      object 
 8   reporterDesc              7037 non-null   object 
 9   flowCode                  7037 non-null   object 
 10  flowDesc                  7037 non-null   object 
 11  partnerCode               7037 non-null   int64  
 12  partnerISO                0 non-null      object 
 13  partnerDesc               7037 non-null   object 
 14  partner2C

In [ ]:
cumul

In [ ]:
suppliers =  comtrade.export_products = comtrade.top_commodities(reporterCode=None,
                                               partnerCode=country_of_interest,
                                               years=years_of_interest, 
                                               cmdCode='AG2',
                                               flowCode='M',  # exports from partners imports
                                           motCode=0,
                                           rank_filter=show_only_top)

KeyboardInterrupt: 

In [ ]:
import pandas as pd

pd.options.display.float_format='{:,.2f}'.format
pd.options.display.max_rows = 1000

cols = ['refYear','flowDesc','cmdDesc','cmdCode','reporterDesc', 'rank_cmd','primaryValue',
        comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD,'perc_year_flow']
export_products[export_products['reporterDesc'].isin(top_export_destinations)].sort_values(['refYear','primaryValue'],ascending=[True, False])[cols]

,refYear,flowDesc,cmdDesc,cmdCode,reporterDesc,rank_cmd,primaryValue,perc_cmd_for_partner,perc_partner_for_cmd,perc_year_flow
195,2020,Import,"Mineral fuels, mineral oils and products of th...",27,China,1,"14,686,950,196.00",0.61,0.66,0.61
2319,2020,Import,"Mineral fuels, mineral oils and products of th...",27,India,1,"2,005,413,125.03",0.08,0.09,0.08
2640,2020,Import,"Mineral fuels, mineral oils and products of th...",27,Thailand,1,"1,250,575,334.49",0.05,0.06,0.05
2680,2020,Import,"Natural, cultured pearls; precious, semi-preci...",71,United Arab Emirates,1,"1,015,602,701.83",0.04,0.72,0.04
2503,2020,Import,"Mineral fuels, mineral oils and products of th...",27,Spain,1,"604,026,342.70",0.03,0.03,0.03
1275,2020,Import,"Mineral fuels, mineral oils and products of th...",27,"Asia, not elsewhere specified",1,"496,824,439.89",0.02,0.02,0.02
710,2020,Import,"Mineral fuels, mineral oils and products of th...",27,France and Monaco,1,"495,407,183.67",0.02,0.02,0.02
2775,2020,Import,"Mineral fuels, mineral oils and products of th...",27,United States of America,1,"482,834,261.00",0.02,0.02,0.02
1282,2020,Import,"Mineral fuels, mineral oils and products of th...",27,Portugal,1,"424,369,024.42",0.02,0.02,0.02
2321,2020,Import,"Mineral fuels, mineral oils and products of th...",27,South Africa,1,"35,607,163.15",0.00,0.00,0.00


In [ ]:
export_products[comtrade.PERC_CMD_IN_PARTNER].sum()

1.9928746283221535

## Principais produtos exportados e destinos

In [ ]:
show_only_top = 5
export_products = comtrade.top_commodities(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='X',
                                           motCode=0,
                                           rank_filter=show_only_top)

TypeError: top_commodities() got an unexpected keyword argument 'extra_cols'

In [ ]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdDesc','rank_partner','partnerDesc',
        comtrade.PERC_PARTNER_IN_CMD, 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER ]

show_only_greater_than = export_products[comtrade.PERC_PARTNER_IN_CMD]>0.01
export_products[show_only_greater_than][cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_partner,partnerDesc,perc_partner_for_cmd,primaryValueFormated,perc_cmd_for_partner
2216,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",1,China,63.93%,"20,022,774,116.55",99.82%
4743,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2,India,9.28%,"2,907,767,269.85",96.66%
5779,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",4,Thailand,3.21%,"1,004,404,810.25",99.96%
2222,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",5,Singapore,2.40%,"751,652,390.95",93.65%
4827,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",6,Spain,2.01%,"628,140,238.31",79.11%
1684,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",7,Italy,2.00%,"625,729,106.51",99.00%
5824,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",9,Chile,1.65%,"516,869,632.99",99.73%
2213,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",8,South Africa,1.56%,"489,628,802.00",94.28%
2343,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",10,Netherlands,1.46%,"456,193,575.29",95.57%
4506,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",12,Indonesia,1.42%,"443,927,852.71",99.93%


## Principais **parceiros** das exportações e produtos exportados

In [ ]:
show_only_top = 5
export_partners = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           cmdCode='AG2',
                                           flowCode='X',
                                           motCode=0, 
                                           rank_filter=5)

In [ ]:
cols = ['reporterDesc','refYear', 'flowDesc','rank_partner',	'partnerDesc','rank_cmd', 'cmdDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]
export_partners[cols].sort_values(['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, 
                                                                                       comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,rank_cmd,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
2216,Angola,2021,Export,1,China,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","20,022,774,116.55",99.82%,63.93%
5170,Angola,2021,Export,1,China,6,"Salt; sulphur; earths, stone; plastering materials, lime and cement","11,961,705.37",0.06%,25.51%
4696,Angola,2021,Export,1,China,5,Wood and articles of wood; wood charcoal,"11,645,905.65",0.06%,23.44%
890,Angola,2021,Export,1,China,13,"Ores, slag and ash","6,616,586.45",0.03%,66.65%
1100,Angola,2021,Export,1,China,18,Aluminium and articles thereof,"966,966.75",0.00%,17.60%
4743,Angola,2021,Export,2,India,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","2,907,767,269.85",96.66%,9.28%
3057,Angola,2021,Export,2,India,2,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin","100,217,986.40",3.33%,6.52%
1864,Angola,2021,Export,2,India,3,"Ships, boats and floating structures","106,755.20",0.00%,0.01%
1254,Angola,2021,Export,2,India,6,"Salt; sulphur; earths, stone; plastering materials, lime and cement","70,361.90",0.00%,0.15%
586,Angola,2021,Export,2,India,73,Pulp of wood or other fibrous cellulosic material; recovered (waste and scrap) paper or paperboard,"46,527.16",0.00%,99.32%


# Importações

## Principais produtos importados

In [ ]:
show_only_top=10
export_products = comtrade.top_commodities(country_of_interest,
                                               partnerCode=0,  # 0 = all partners
                                               years=years_of_interest, 
                                               flowCode='M',
                                           motCode=0,
                                           rank_filter=10)


In [ ]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdCode','cmdDesc', 'primaryValueFormated',
        'partnerDesc',
        comtrade.PERC_CMD_IN_PARTNER ]
export_products[cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdCode,cmdDesc,primaryValueFormated,partnerDesc,perc_cmd_for_partner
402,Angola,2021,Import,1,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","1,886,757,547.56",World,16.58%
230,Angola,2021,Import,2,84,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","1,809,939,647.27",World,15.91%
126,Angola,2021,Import,3,87,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","779,123,345.04",World,6.85%
367,Angola,2021,Import,4,85,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","776,747,585.90",World,6.83%
161,Angola,2021,Import,5,10,Cereals,"584,957,170.07",World,5.14%
27,Angola,2021,Import,6,73,Iron or steel articles,"536,792,046.75",World,4.72%
362,Angola,2021,Import,7,39,Plastics and articles thereof,"405,040,625.98",World,3.56%
240,Angola,2021,Import,8,02,Meat and edible meat offal,"348,250,700.18",World,3.06%
322,Angola,2021,Import,9,30,Pharmaceutical products,"322,068,826.04",World,2.83%
169,Angola,2021,Import,10,15,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","311,937,457.18",World,2.74%


## Principais produtos importados e origens

In [ ]:
show_only_top = 10
export_products = comtrade.top_commodities(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='M',
                                           cmdCode='AG2',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [ ]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdDesc','rank_partner','partnerDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_products[comtrade.PERC_PARTNER_IN_CMD] > 0.05  # show only partners with more than 5% of trade
export_products[filter_relevant_partners][cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_partner,partnerDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
3954,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",4,Togo,"687,679,076.53",100.00%,36.45%
13156,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",8,Netherlands,"318,111,982.79",73.62%,16.86%
13215,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",3,India,"291,213,622.52",39.05%,15.43%
3776,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",17,Malta,"184,307,230.08",99.00%,9.77%
4069,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",11,Belgium,"113,736,727.71",29.24%,6.03%
7421,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",1,China,"294,737,063.36",17.45%,16.28%
7026,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",2,Portugal,"219,815,557.96",16.19%,12.14%
1351,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",9,Italy,"206,406,850.29",48.33%,11.40%
7219,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",6,United Kingdom,"165,889,250.91",30.75%,9.17%
1111,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",7,United States of America,"159,642,149.65",30.52%,8.82%


### Principal origem das importações

In [ ]:
show_ony_top=20
top_partners = comtrade.top_partners(country_of_interest,years_of_interest,
                                     flowCode='M', 
                                    rank_filter=show_ony_top)

In [ ]:
cols=['reporterDesc','refYear','flowDesc','rank_partner','partnerDesc','primaryValueFormated', comtrade.PERC_PARTNER_IN_CMD]
top_partners[cols].sort_values(by=['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,primaryValueFormated,perc_partner_for_cmd
258,Angola,2021,Import,1,China,"1,688,904,641.34",14.84%
95,Angola,2021,Import,2,Portugal,"1,357,979,633.71",11.93%
323,Angola,2021,Import,3,India,"745,723,527.22",6.55%
319,Angola,2021,Import,4,Togo,"687,689,058.08",6.04%
495,Angola,2021,Import,5,Brazil,"546,044,463.65",4.80%
610,Angola,2021,Import,6,United Kingdom,"539,546,476.65",4.74%
251,Angola,2021,Import,7,United States of America,"523,006,089.83",4.60%
455,Angola,2021,Import,8,Netherlands,"432,100,821.51",3.80%
192,Angola,2021,Import,9,Italy,"427,046,959.44",3.75%
36,Angola,2021,Import,10,France and Monaco,"411,360,213.52",3.62%


## Principais **parceiros** das importações e produtos importados

In [ ]:
show_only_top = 10
export_partners = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           cmdCode='AG2',
                                           flowCode='M',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [ ]:
cols = ['reporterDesc','refYear', 'flowDesc','rank_partner','partnerDesc', 'cmdDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_partners[comtrade.PERC_PARTNER_IN_CMD] > 0.0  # show only partners with more than 5% of trade
export_partners[filter_relevant_partners][cols].sort_values(['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
7421,Angola,2021,Import,1,China,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","294,737,063.36",17.45%,16.28%
1837,Angola,2021,Import,1,China,"Textiles, made up articles; sets; worn clothing and worn textile articles; rags","207,543,201.07",12.29%,80.04%
5764,Angola,2021,Import,1,China,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","188,027,402.48",11.13%,24.21%
3683,Angola,2021,Import,1,China,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","170,241,136.72",10.08%,21.85%
12591,Angola,2021,Import,1,China,Iron and steel,"94,179,850.47",5.58%,64.83%
5371,Angola,2021,Import,1,China,Plastics and articles thereof,"90,509,512.27",5.36%,22.35%
11724,Angola,2021,Import,1,China,Glass and glassware,"62,248,517.42",3.69%,75.37%
6889,Angola,2021,Import,1,China,Iron or steel articles,"60,344,496.23",3.57%,11.24%
6008,Angola,2021,Import,1,China,Rubber and articles thereof,"58,498,228.59",3.46%,43.76%
11490,Angola,2021,Import,1,China,Pharmaceutical products,"48,373,928.61",2.86%,15.02%


## Análise de simetrias nos dados

In [ ]:


partnerA = country_of_interest


### Principais destinos de exportação reportados pelo país

In [ ]:
show_only_top = 30
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='X',
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=None,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     customsCode=None,
                     echo_url=False
                     )
cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>100000][cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format}) 


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
348,Angola,China,World,2021,Export,"20,058,426,271.91",20058426271.912998,58.19%,All modes of transport,True,All customs procedure codes
211,Angola,India,World,2021,Export,"3,008,390,595.07",3008390595.072000,8.73%,All modes of transport,True,All customs procedure codes
288,Angola,United Arab Emirates,World,2021,Export,"1,399,449,578.20",1399449578.198000,4.06%,All modes of transport,True,All customs procedure codes
165,Angola,Thailand,World,2021,Export,"1,004,833,529.77",1004833529.770000,2.91%,All modes of transport,True,All customs procedure codes
46,Angola,Singapore,World,2021,Export,"802,659,408.76",802659408.761000,2.33%,All modes of transport,True,All customs procedure codes
109,Angola,Spain,World,2021,Export,"793,986,998.99",793986998.994000,2.30%,All modes of transport,True,All customs procedure codes
67,Angola,Italy,World,2021,Export,"632,025,040.54",632025040.539000,1.83%,All modes of transport,True,All customs procedure codes
279,Angola,South Africa,World,2021,Export,"519,358,721.33",519358721.333000,1.51%,All modes of transport,True,All customs procedure codes
70,Angola,Chile,World,2021,Export,"518,264,559.23",518264559.233000,1.50%,All modes of transport,True,All customs procedure codes
63,Angola,Netherlands,World,2021,Export,"477,332,019.79",477332019.785000,1.38%,All modes of transport,True,All customs procedure codes


### Principais destinos de exportação segundo o registo da importações do resto do mundo

In [ ]:
show_only_top=30
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='M',
                     cmdCode='TOTAL',
                     reporterCode=None,
                     partnerCode=partnerA,
                     partner2Code=0,
                     motCode = 0,
                     # customsCode=0,
                     period=years_of_interest,
                     echo_url=False
                     )
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>1000][cols].sort_values(by=['refYear','primaryValue','reporterDesc',], ascending=[True,False,True]).head(show_only_top).style.format({'perc': '{:,.2%}'.format}) 

,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
29,China,Angola,World,2021,Import,"20,908,231,317.00",20908231317.000000,62.45%,All modes of transport,True,All customs procedure codes
324,India,Angola,World,2021,Import,"2,315,947,568.41",2315947568.408000,6.92%,All modes of transport,True,All customs procedure codes
388,United Arab Emirates,Angola,World,2021,Import,"1,685,701,656.52",1685701656.518000,5.04%,All modes of transport,True,All customs procedure codes
385,Thailand,Angola,World,2021,Import,"1,607,756,231.51",1607756231.510000,4.80%,All modes of transport,True,All customs procedure codes
413,United States of America,Angola,World,2021,Import,"1,089,250,402.00",1089250402.000000,3.25%,All modes of transport,True,All customs procedure codes
325,Singapore,Angola,World,2021,Import,"752,550,761.40",752550761.398000,2.25%,All modes of transport,True,All customs procedure codes
355,Spain,Angola,World,2021,Import,"739,484,782.77",739484782.768000,2.21%,All modes of transport,True,All customs procedure codes
5,Belgium,Angola,World,2021,Import,"615,024,126.87",615024126.871000,1.84%,All modes of transport,True,All customs procedure codes
28,Chile,Angola,World,2021,Import,"537,823,552.96",537823552.960000,1.61%,All modes of transport,True,All customs procedure codes
147,Indonesia,Angola,World,2021,Import,"463,466,072.00",463466072.000000,1.38%,All modes of transport,True,All customs procedure codes


## Análise de simetrias nos dados com um país específico

In [ ]:
partnerB = comtrade.COUNTRY_CODES_REVERSE['China']
flowCode = 'M'
flowDesc = comtrade.FLOWS_CODES[flowCode]


phrase = f"{comtrade.COUNTRY_CODES.get(partnerA)} {flowDesc} to {comtrade.COUNTRY_CODES.get(partnerB)} reported by {comtrade.COUNTRY_CODES.get(partnerA)}" \
 if flowCode == 'X' else f"{comtrade.COUNTRY_CODES.get(partnerA)} {flowDesc} from {comtrade.COUNTRY_CODES.get(partnerB)} reported by {comtrade.COUNTRY_CODES.get(partnerA)}"

cmd_detail = 'AG4'
show_only_top = 10

cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','cmdDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 

In [ ]:
df_import_rep_a = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode=cmd_detail,
                     reporterCode=partnerA,
                     partnerCode=partnerB,
                     partner2Code=0,
                     period=2020,
                     motCode = 0,
                     echo_url=False
                     )


In [ ]:
if df_import_rep_a is not None:
    total_import = df_import_rep_a['primaryValue'].sum()
    df_import_rep_a['perc'] = df_import_rep_a['primaryValue']/total_import
    print(phrase)
    result = df_import_rep_a[cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print("No",phrase)
    result = ''

result

Angola Import from China reported by Angola


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,cmdDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
1021,Angola,China,World,2020,Import,"Rail locomotives; (other than those of heading no. 8601), locomotive tenders","106,385,404.28",106385404.277000,7.42%,All modes of transport,True,All customs procedure codes
1382,Angola,China,World,2020,Import,"New pneumatic tyres, of rubber","50,669,463.76",50669463.757000,3.53%,All modes of transport,True,All customs procedure codes
1087,Angola,China,World,2020,Import,nan,"48,169,359.41",48169359.409000,3.36%,All modes of transport,True,All customs procedure codes
1722,Angola,China,World,2020,Import,"Iron or non-alloy steel; flat-rolled products, width 600mm or more, clad, plated or coated","40,716,106.16",40716106.155000,2.84%,All modes of transport,True,All customs procedure codes
508,Angola,China,World,2020,Import,"Electric transformers, static converters (e.g. rectifiers) and inductors","39,653,621.92",39653621.919000,2.76%,All modes of transport,True,All customs procedure codes
2340,Angola,China,World,2020,Import,"Iron or non-alloy steel; flat-rolled products of a width of 600mm or more, hot-rolled, not clad, plated or coated","39,392,570.45",39392570.454000,2.75%,All modes of transport,True,All customs procedure codes
381,Angola,China,World,2020,Import,"Telephone sets, including smartphones and other telephones for cellular/wireless networks; other apparatus for the transmission or reception of voice, images or other data (including wired/wireless networks), excluding items of 8443, 8525, 8527, or 8528","36,628,189.17",36628189.175000,2.55%,All modes of transport,True,All customs procedure codes
28,Angola,China,World,2020,Import,"Medicaments; (not goods of heading no. 3002, 3005 or 3006) consisting of mixed or unmixed products for therapeutic or prophylactic use, put up in measured doses (incl. those in the form of transdermal admin. systems) or packed for retail sale","29,087,289.30",29087289.296000,2.03%,All modes of transport,True,All customs procedure codes
2147,Angola,China,World,2020,Import,"Air conditioning machines; comprising a motor driven fan and elements for changing the temperature and humidity, including those machines in which the humidity cannot be separately regulated","22,047,690.54",22047690.539000,1.54%,All modes of transport,True,All customs procedure codes
2671,Angola,China,World,2020,Import,"Clock, watch and similar glasses, glasses for non-corrective or corrective spectacles, curved, bent, hallowed etc, not optically worked; hollow glass spheres and their segments for manufacture","21,833,158.76",21833158.762000,1.52%,All modes of transport,True,All customs procedure codes


In [ ]:
# invertemos

print(years_of_interest, partnerA, partnerB)

flowCodeInv = 'X' if flowCode == 'M' else 'M'
flowDesc = comtrade.FLOWS_CODES[flowCodeInv]


phrase = f"{comtrade.COUNTRY_CODES.get(partnerB)} {flowDesc} to {comtrade.COUNTRY_CODES.get(partnerA)} reported by {comtrade.COUNTRY_CODES.get(partnerB)}" if flowCodeInv == 'X'\
    else f"{comtrade.COUNTRY_CODES.get(partnerB)} {flowDesc} from {comtrade.COUNTRY_CODES.get(partnerA)} reported by {comtrade.COUNTRY_CODES.get(partnerB)}"


print(phrase)

df_import_rep_b = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCodeInv,
                     cmdCode=cmd_detail,
                     reporterCode=partnerB,
                     partnerCode=partnerA,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
if df_import_rep_b is not None:
    total_import = df_import_rep_b['primaryValue'].sum()
    df_import_rep_b['perc'] = df_import_rep_b['primaryValue']/total_import
    result = df_import_rep_b[cols].sort_values(by=['refYear','reporterDesc','primaryValue'], ascending=[True,True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print("No",phrase)
    result = ''
result

2021 24 156
China Export to Angola reported by China


ReadTimeout: HTTPSConnectionPool(host='comtradeapi.un.org', port=443): Read timed out. (read timeout=10)